In [1]:
from eratos.creds import AccessTokenCreds
from eratos.adapter import Adapter
import eratos.climate as eratosClimate
import eratos.helpers as helpers
import os
import yaml
from yaml.loader import SafeLoader
import json
import pprint
import datetime
import shapely
from shapely import wkt, geometry
from datetime import timezone
from keplergl import KeplerGl
import geopandas as gpd
import numpy as np
import datetime
import json
import pandas as pd
import numpy as np
from datetime import  timezone
from datetime import date
import geopandas as gpd
from shapely.geometry import box
from shapely import wkt
import datetime
import os

ERATOS_KEY = os.environ['ERATOS_KEY']
ERATOS_SECRET = os.environ['ERATOS_SECRET']

ecreds = AccessTokenCreds(
  ERATOS_KEY,
  ERATOS_SECRET
)
eadapter = Adapter(ecreds)

In [24]:
def get_forecast_data_for_points(ern: str, point_list: str):

    # Parse the date range using the iso8601 format
    e_data = eadapter.Resource(ern=ern)
    #access the gridded data via the gridded data adapter:
    gridded_e_data = e_data.data().gapi()
         # Load Point strings as Shapely Points
    wkt_list = []
    for point in (point_list):
        loc_shape = wkt.loads(point)
        if type(loc_shape) is not geometry.Point:
            raise ValueError('value inside point_list should be a WKT point')
        loc= [loc_shape.y, loc_shape.x]
        wkt_list.append(loc)


    data_query_array = gridded_e_data.get_point_slices(gridded_e_data.get_key_variables()[0], 'SPP', pts=wkt_list, starts=[0], ends=[-1],strides =  [1])

    times = []
    for unix_time in gridded_e_data.get_subset_as_array('time'):

        date = datetime.datetime.fromtimestamp(unix_time)

        # Format the datetime object as an ISO 8601 string
        date_8601 = date.isoformat()
        times.append(date_8601)

    return data_query_array,times

def get_forecast_multiple_data_at_point(ern_list, point:str):
     data = {}
     times = None
     for ern in ern_list:
         e_data = eadapter.Resource(ern=ern)
         gridded_e_data = e_data.data().gapi()
         if times is None:
             times = gridded_e_data.get_subset_as_array('time')
             data['times'] = times
         variable = gridded_e_data.get_key_variables()[0]

         p = wkt.loads(point)
         pt = [p.y, p.x]
         data[variable] = gridded_e_data.get_point_slices(
           variable, 'SPP', pts = [pt], starts = [0], ends=[-1], strides=  [1]
         )
     return data
     data = pd.DataFrame(data)
     data.times = pd.to_datetime(data.times)
     return data


In [25]:
ern_list = [
    'ern:e-pn.io:resource:eratos.blocks.bom.adfd.3hourlyskycoverforecastau6km',
    'ern:e-pn.io:resource:eratos.blocks.bom.adfd.dailymintempforecastau6km',
    'ern:e-pn.io:resource:eratos.blocks.bom.adfd.dewpointtempforecastau6km'
]
location = 'POINT(141.181121 -34.147659)'


In [27]:
data = get_forecast_multiple_data_at_point(ern_list, location)

In [28]:
data

{'times': array([1.6709544e+09, 1.6709652e+09, 1.6709760e+09, 1.6709868e+09,
        1.6709976e+09, 1.6710084e+09, 1.6710192e+09, 1.6710300e+09,
        1.6710408e+09, 1.6710516e+09, 1.6710624e+09, 1.6710732e+09,
        1.6710840e+09, 1.6710948e+09, 1.6711056e+09, 1.6711164e+09,
        1.6711272e+09, 1.6711380e+09, 1.6711488e+09, 1.6711596e+09,
        1.6711704e+09, 1.6711812e+09, 1.6711920e+09, 1.6712028e+09,
        1.6712136e+09, 1.6712244e+09, 1.6712352e+09, 1.6712460e+09,
        1.6712568e+09, 1.6712676e+09, 1.6712784e+09, 1.6712892e+09,
        1.6713000e+09, 1.6713108e+09, 1.6713216e+09, 1.6713324e+09,
        1.6713432e+09, 1.6713540e+09, 1.6713648e+09, 1.6713756e+09,
        1.6713864e+09, 1.6713972e+09, 1.6714080e+09, 1.6714188e+09,
        1.6714296e+09, 1.6714404e+09, 1.6714512e+09, 1.6714620e+09,
        1.6714728e+09, 1.6714836e+09, 1.6714944e+09, 1.6715052e+09,
        1.6715160e+09, 1.6715268e+09, 1.6715376e+09]),
 'Sky_SFC': array([[ 0., 17., 50., 46., 23., 18., 38

In [14]:
cloud_cover_data.get_key_variables()

['Sky_SFC']

In [7]:
location = 'POINT(141.181121 -34.147659)'
p =wkt.loads(location)

In [11]:
cloud_cover_data.get_point_slices('Sky_SFC', )